<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 21: Standards Integration - How ISO 26262, 21448, and 21434 Work Together

**Session 5: Standards Integration and Deployment**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/master/08_Advanced_Topics/notebooks/21_Standards_Integration.ipynb)

**Author:** Milin Patel

---

## Learning Objectives

- Understand how ISO 26262, ISO 21448, and ISO/SAE 21434 complement each other
- Learn which standard applies to which hazard type
- Resolve conflicts between standards
- Perform combined HARA + SOTIF + TARA analysis
- Create integrated safety case
- Navigate real-world compliance workflow

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

print("✓ Setup complete")

---

## 1. The Three Standards: What Each Covers

### ISO 26262: Functional Safety

**Scope:** Failures due to **random hardware faults** or **systematic software bugs**

**Examples:**
- Sensor hardware fails (camera stopped working)
- Software bug causes wrong calculation
- ECU memory corruption

**Question it answers:** "What if a component fails?"

**Key process:** HARA (Hazard Analysis and Risk Assessment) → ASIL

### ISO 21448: SOTIF (Safety of Intended Functionality)

**Scope:** Failures when system works **as designed** but design is insufficient

**Examples:**
- Perception model cannot detect dark clothing at night
- Sensor physically limited (camera can't see through fog)
- Algorithm limitation (cannot handle rare edge case)

**Question it answers:** "What if the design itself is flawed?"

**Key process:** Performance limitation + triggering condition analysis

### ISO/SAE 21434: Cybersecurity

**Scope:** Failures due to **intentional attacks**

**Examples:**
- Hacker spoofs GNSS signal
- Attacker injects false V2X messages
- Malware corrupts perception data

**Question it answers:** "What if someone attacks the system?"

**Key process:** TARA (Threat Analysis and Risk Assessment) → CAL

### Visual Summary

In [ ]:
# Venn diagram showing standards overlap
fig, ax = plt.subplots(figsize=(10, 8))

# Three circles
from matplotlib.patches import Circle
circle1 = Circle((0.3, 0.5), 0.25, alpha=0.3, color='blue', label='ISO 26262\n(Random/Systematic Faults)')
circle2 = Circle((0.5, 0.7), 0.25, alpha=0.3, color='green', label='ISO 21448\n(Performance Limits)')
circle3 = Circle((0.7, 0.5), 0.25, alpha=0.3, color='red', label='ISO/SAE 21434\n(Attacks)')

ax.add_patch(circle1)
ax.add_patch(circle2)
ax.add_patch(circle3)

# Labels
ax.text(0.2, 0.4, 'Hardware\nfailure', ha='center', fontsize=9, fontweight='bold')
ax.text(0.5, 0.85, 'OOD\nscenario', ha='center', fontsize=9, fontweight='bold')
ax.text(0.8, 0.4, 'V2X\nspoofing', ha='center', fontsize=9, fontweight='bold')

# Overlaps
ax.text(0.4, 0.6, 'SW bug +\nOOD', ha='center', fontsize=8, style='italic')
ax.text(0.6, 0.6, 'Attack\nexploits\nlimit', ha='center', fontsize=8, style='italic')
ax.text(0.5, 0.5, 'ALL THREE\napply', ha='center', fontsize=8, bbox=dict(boxstyle='round', facecolor='yellow'))

ax.set_xlim(0, 1)
ax.set_ylim(0.2, 1)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('How Standards Overlap (Venn Diagram)', fontsize=14, fontweight='bold')

plt.legend(loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.05))
plt.tight_layout()
plt.show()

print("\n📊 Key Insight: Most real hazards require MULTIPLE standards!")

---

## 2. Decision Tree: Which Standard Applies When?

A systematic approach to determine which standards you need to apply for a given hazard.

### Decision Tree Algorithm

```
START: You identified a potential hazard
│
├─ Q1: Is this hazard caused by an intentional attack?
│ │
│ YES ─→ ISO/SAE 21434 (TARA) ─→ Continue to Q2
│ │
│ NO ─→ Continue to Q2
│
├─ Q2: Can this hazard occur even when all components work correctly?
│ │
│ YES ─→ ISO 21448 (SOTIF) ─→ Continue to Q3
│ │
│ NO ─→ Continue to Q3
│
├─ Q3: Can this hazard be caused by component failure?
│ │
│ YES ─→ ISO 26262 (HARA)
│ │
│ NO ─→ No standard? (Unlikely - re-examine!)
│
END: Apply all selected standards in parallel
```

### Key Rule: **It's NOT exclusive!**

Most hazards require **multiple standards** because:
- A hardware failure (26262) can make the system vulnerable to an attack (21434)
- A performance limitation (21448) might not be detected if the monitor fails (26262)
- An attack (21434) can push the system into an unknown scenario (21448)

In [ ]:
# Interactive decision tree helper
def which_standards_apply(intentional_attack, works_as_designed_failure, component_failure):
    """
    Determine which standards apply to a given hazard.
    
    Parameters:
    - intentional_attack: bool - Can this be caused by an attacker?
    - works_as_designed_failure: bool - Can this happen even when components work correctly?
    - component_failure: bool - Can this be caused by a component failing?
    """
    standards = []
    
    if intentional_attack:
        standards.append("ISO/SAE 21434 (Cybersecurity)")
    
    if works_as_designed_failure:
        standards.append("ISO 21448 (SOTIF)")
    
    if component_failure:
        standards.append("ISO 26262 (Functional Safety)")
    
    if not standards:
        standards.append("⚠️ No standard selected - please re-examine the hazard!")
    
    return standards

# Example hazards
hazards = [
    {
        "name": "Camera fails to detect pedestrian in fog",
        "intentional_attack": False,
        "works_as_designed": True,  # Physical sensor limitation
        "component_failure": False,
    },
    {
        "name": "Radar sensor hardware failure",
        "intentional_attack": False,
        "works_as_designed": False,
        "component_failure": True,
    },
    {
        "name": "Attacker spoofs GNSS signal",
        "intentional_attack": True,
        "works_as_designed": False,
        "component_failure": False,
    },
    {
        "name": "ML model cannot detect dark cyclist at night",
        "intentional_attack": False,
        "works_as_designed": True,
        "component_failure": False,
    },
    {
        "name": "Software bug causes wrong brake calculation + fog scenario",
        "intentional_attack": False,
        "works_as_designed": True,
        "component_failure": True,
    },
]

# Apply decision tree
results = []
for hazard in hazards:
    applicable = which_standards_apply(
        hazard["intentional_attack"],
        hazard["works_as_designed"],
        hazard["component_failure"]
    )
    results.append({
        "Hazard": hazard["name"],
        "Standards Required": ", ".join(applicable),
        "Count": len([s for s in applicable if "⚠️" not in s])
    })

df_decision = pd.DataFrame(results)
print("=" * 100)
print("DECISION TREE RESULTS: Which Standards Apply?")
print("=" * 100)
display(df_decision)

# Visualize standards frequency
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count of standards per hazard
counts = df_decision['Count'].value_counts().sort_index()
ax1.bar(counts.index, counts.values, color=['skyblue', 'orange', 'green'])
ax1.set_xlabel('Number of Standards Required', fontsize=11, fontweight='bold')
ax1.set_ylabel('Number of Hazards', fontsize=11, fontweight='bold')
ax1.set_title('How Many Standards per Hazard?', fontsize=12, fontweight='bold')
ax1.set_xticks([1, 2, 3])
ax1.grid(axis='y', alpha=0.3)

# Frequency of each standard
all_standards = []
for hazard in hazards:
    all_standards.extend(which_standards_apply(
        hazard["intentional_attack"],
        hazard["works_as_designed"],
        hazard["component_failure"]
    ))

standard_names = [s.split(' (')[0] for s in all_standards if "⚠️" not in s]
standard_counts = pd.Series(standard_names).value_counts()

colors = {'ISO/SAE 21434': 'red', 'ISO 21448': 'green', 'ISO 26262': 'blue'}
bar_colors = [colors.get(s, 'gray') for s in standard_counts.index]

ax2.barh(standard_counts.index, standard_counts.values, color=bar_colors, alpha=0.7)
ax2.set_xlabel('Frequency', fontsize=11, fontweight='bold')
ax2.set_title('Which Standards Are Most Common?', fontsize=12, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Decision tree analysis complete!")

---

## 3. Combined HARA + SOTIF + TARA Workflow

When a hazard requires multiple standards, you need an **integrated workflow** that addresses all perspectives simultaneously.

### Step-by-Step Integrated Process

```
1. IDENTIFY HAZARD
 ↓
2. CLASSIFY ROOT CAUSE(S)
 - Component failure? → ISO 26262
 - Performance limit? → ISO 21448
 - Attack vector? → ISO/SAE 21434
 ↓
3. PARALLEL ANALYSIS (for each applicable standard)
 ┌─────────────────┬──────────────────┬─────────────────┐
 │ ISO 26262 (HARA)│ ISO 21448 (SOTIF)│ ISO 21434 (TARA)│
 ├─────────────────┼──────────────────┼─────────────────┤
 │ S, E, C → ASIL │ Scenario + Perf │ Impact, Attack │
 │ │ Insufficiency │ Feasibility→CAL │
 └─────────────────┴──────────────────┴─────────────────┘
 ↓
4. INTEGRATE RESULTS
 - Take HIGHEST severity level
 - Combine safety requirements
 - Resolve conflicts
 ↓
5. DEFINE SAFETY GOALS & REQUIREMENTS
 - Must satisfy ALL applicable standards
 ↓
6. VERIFY & VALIDATE
 - Each standard has its own V&V requirements
 - Integration testing for combined scenarios
```

### Key Principle: **Conservative Approach**

When multiple standards apply:
- **Use the HIGHEST severity level** (most conservative)
- **Combine ALL requirements** (not pick one)
- **Test for COMBINED scenarios** (e.g., fog + hardware failure)

In [ ]:
# Example: Combined analysis for a complex hazard
class IntegratedSafetyAnalysis:
    """Perform combined HARA + SOTIF + TARA analysis"""
    
    def __init__(self, hazard_description):
        self.hazard = hazard_description
        self.analyses = {}
    
    def hara_analysis(self, severity, exposure, controllability):
        """ISO 26262 HARA"""
        asil_map = {
            (3, 4, 3): "ASIL D",
            (3, 4, 2): "ASIL D",
            (3, 3, 3): "ASIL D",
            (3, 3, 2): "ASIL C",
            (2, 4, 3): "ASIL C",
            (2, 3, 3): "ASIL B",
            (1, 4, 3): "ASIL B",
        }
        asil = asil_map.get((severity, exposure, controllability), "ASIL A")
        
        self.analyses['ISO 26262'] = {
            'Severity': severity,
            'Exposure': exposure,
            'Controllability': controllability,
            'ASIL': asil
        }
        return asil
    
    def sotif_analysis(self, performance_limit, triggering_condition_freq):
        """ISO 21448 SOTIF"""
        if performance_limit == "Critical" and triggering_condition_freq == "High":
            risk_level = "HIGH"
        elif performance_limit == "Critical" or triggering_condition_freq == "High":
            risk_level = "MEDIUM"
        else:
            risk_level = "LOW"
        
        self.analyses['ISO 21448'] = {
            'Performance Limitation': performance_limit,
            'Triggering Condition Frequency': triggering_condition_freq,
            'SOTIF Risk': risk_level
        }
        return risk_level
    
    def tara_analysis(self, impact_rating, attack_feasibility):
        """ISO/SAE 21434 TARA"""
        # CAL = Cybersecurity Assurance Level
        cal_map = {
            ("Severe", "High"): "CAL 4",
            ("Severe", "Medium"): "CAL 3",
            ("Major", "High"): "CAL 3",
            ("Major", "Medium"): "CAL 2",
            ("Moderate", "High"): "CAL 2",
        }
        cal = cal_map.get((impact_rating, attack_feasibility), "CAL 1")
        
        self.analyses['ISO/SAE 21434'] = {
            'Impact Rating': impact_rating,
            'Attack Feasibility': attack_feasibility,
            'CAL': cal
        }
        return cal
    
    def integrate(self):
        """Integrate all analyses"""
        if not self.analyses:
            return "No analyses performed yet"
        
        # Map severity levels to numeric values for comparison
        severity_map = {
            'ASIL D': 4, 'ASIL C': 3, 'ASIL B': 2, 'ASIL A': 1,
            'HIGH': 4, 'MEDIUM': 3, 'LOW': 2,
            'CAL 4': 4, 'CAL 3': 3, 'CAL 2': 2, 'CAL 1': 1
        }
        
        max_severity = 0
        max_standard = None
        max_level = None
        
        for standard, analysis in self.analyses.items():
            if 'ASIL' in analysis:
                level = analysis['ASIL']
            elif 'SOTIF Risk' in analysis:
                level = analysis['SOTIF Risk']
            elif 'CAL' in analysis:
                level = analysis['CAL']
            else:
                continue
            
            severity = severity_map.get(level, 0)
            if severity > max_severity:
                max_severity = severity
                max_standard = standard
                max_level = level
        
        return {
            'Integrated Severity': max_level,
            'Driving Standard': max_standard,
            'All Standards': list(self.analyses.keys())
        }
    
    def generate_report(self):
        """Generate integrated report"""
        print("=" * 90)
        print(f"INTEGRATED SAFETY ANALYSIS: {self.hazard}")
        print("=" * 90)
        
        for standard, analysis in self.analyses.items():
            print(f"\n{standard}:")
            for key, value in analysis.items():
                print(f"  {key}: {value}")
        
        print("\n" + "-" * 90)
        print("INTEGRATION RESULT:")
        integration = self.integrate()
        for key, value in integration.items():
            print(f"  {key}: {value}")
        print("=" * 90)

# Example: Analyze a complex hazard
analysis = IntegratedSafetyAnalysis(
    "AEB fails to brake for pedestrian in night-time fog with spoofed radar"
)

# ISO 26262: Component failure aspect (radar spoofing could be seen as sensor failure)
asil = analysis.hara_analysis(severity=3, exposure=4, controllability=3)

# ISO 21448: Performance limitation (perception in fog)
sotif_risk = analysis.sotif_analysis(
    performance_limit="Critical",
    triggering_condition_freq="High"
)

# ISO/SAE 21434: Cybersecurity aspect (radar spoofing)
cal = analysis.tara_analysis(
    impact_rating="Severe",
    attack_feasibility="Medium"
)

# Generate integrated report
analysis.generate_report()

# Create summary table
summary_data = []
for standard, data in analysis.analyses.items():
    if 'ASIL' in data:
        level = data['ASIL']
    elif 'SOTIF Risk' in data:
        level = data['SOTIF Risk']
    else:
        level = data['CAL']
    
    summary_data.append({
        'Standard': standard,
        'Result': level,
        'Key Metric': list(data.keys())[-1]
    })

df_summary = pd.DataFrame(summary_data)
print("\n")
display(df_summary)

print("\n✓ Integrated analysis complete!")

---

## 4. Real Example: AEB (Automatic Emergency Braking) Full Analysis

Let's perform a complete integrated safety analysis for an AEB system, considering all three standards.

### System Description

**Function:** Detect obstacles and automatically brake to prevent collision

**Components:**
- Camera (front-facing, 60m range)
- Radar (77 GHz, 200m range)
- LiDAR (128-line, 100m range)
- Sensor fusion module
- Brake actuator
- ECU running detection algorithm

### Hazards to Analyze

We'll analyze multiple hazards that the AEB system faces:

1. **H1:** Fail to brake for pedestrian (functional failure)
2. **H2:** Cannot detect pedestrian in fog (performance limit)
3. **H3:** Attacker spoofs radar to show false obstacle (cyber attack)
4. **H4:** Combined: Radar hardware fails + heavy rain + GPS jamming

In [ ]:
# Comprehensive AEB Safety Analysis
aeb_hazards = []

# H1: Fail to brake for pedestrian (ISO 26262)
h1 = IntegratedSafetyAnalysis("H1: Brake actuator fails to respond")
h1.hara_analysis(severity=3, exposure=4, controllability=3)  # ASIL D
aeb_hazards.append(("H1", h1))

# H2: Cannot detect pedestrian in fog (ISO 21448)
h2 = IntegratedSafetyAnalysis("H2: Camera cannot see pedestrian in dense fog")
h2.sotif_analysis(performance_limit="Critical", triggering_condition_freq="High")
aeb_hazards.append(("H2", h2))

# H3: Attacker spoofs radar (ISO/SAE 21434)
h3 = IntegratedSafetyAnalysis("H3: Radar spoofing attack shows false obstacle")
h3.tara_analysis(impact_rating="Severe", attack_feasibility="Medium")
aeb_hazards.append(("H3", h3))

# H4: Combined scenario (ALL THREE)
h4 = IntegratedSafetyAnalysis("H4: Radar fails + Heavy rain reduces visibility + GPS jamming")
h4.hara_analysis(severity=3, exposure=3, controllability=3)  # Hardware failure
h4.sotif_analysis(performance_limit="Critical", triggering_condition_freq="High")  # Rain
h4.tara_analysis(impact_rating="Severe", attack_feasibility="High")  # GPS jamming
aeb_hazards.append(("H4", h4))

# Generate reports for all hazards
for hazard_id, hazard in aeb_hazards:
    print("\n\n")
    hazard.generate_report()
    print("\n")

# Create comprehensive summary table
all_results = []
for hazard_id, hazard_obj in aeb_hazards:
    integration = hazard_obj.integrate()
    
    row = {
        'Hazard ID': hazard_id,
        'Description': hazard_obj.hazard[:50] + "...",
        'Standards Applied': len(hazard_obj.analyses),
        'Final Severity': integration.get('Integrated Severity', 'N/A'),
        'Driving Standard': integration.get('Driving Standard', 'N/A')
    }
    
    # Add individual standard results
    for std in ['ISO 26262', 'ISO 21448', 'ISO/SAE 21434']:
        if std in hazard_obj.analyses:
            analysis = hazard_obj.analyses[std]
            if 'ASIL' in analysis:
                row[std] = analysis['ASIL']
            elif 'SOTIF Risk' in analysis:
                row[std] = analysis['SOTIF Risk']
            elif 'CAL' in analysis:
                row[std] = analysis['CAL']
        else:
            row[std] = '-'
    
    all_results.append(row)

df_aeb = pd.DataFrame(all_results)
print("\n" + "=" * 120)
print("AEB SYSTEM: COMPREHENSIVE HAZARD ANALYSIS SUMMARY")
print("=" * 120)
display(df_aeb)

# Visualize the AEB analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Standards coverage per hazard
hazard_ids = [h[0] for h in aeb_hazards]
std_counts = [len(h[1].analyses) for h in aeb_hazards]
colors_bar = ['blue' if c == 1 else 'orange' if c == 2 else 'red' for c in std_counts]

ax1.bar(hazard_ids, std_counts, color=colors_bar, alpha=0.7, edgecolor='black')
ax1.set_ylabel('Number of Standards', fontsize=11, fontweight='bold')
ax1.set_xlabel('Hazard ID', fontsize=11, fontweight='bold')
ax1.set_title('Standards Coverage per AEB Hazard', fontsize=12, fontweight='bold')
ax1.set_ylim(0, 4)
ax1.grid(axis='y', alpha=0.3)

# 2. Severity distribution
severity_values = {
    'ASIL D': 4, 'ASIL C': 3, 'ASIL B': 2, 'ASIL A': 1,
    'HIGH': 4, 'MEDIUM': 3, 'LOW': 2,
    'CAL 4': 4, 'CAL 3': 3, 'CAL 2': 2, 'CAL 1': 1
}

severities = [severity_values.get(h[1].integrate().get('Integrated Severity', 'LOW'), 1) 
              for h in aeb_hazards]
colors_sev = ['darkred' if s == 4 else 'orange' if s == 3 else 'yellow' for s in severities]

ax2.barh(hazard_ids, severities, color=colors_sev, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Severity Level (1-4)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Hazard ID', fontsize=11, fontweight='bold')
ax2.set_title('Integrated Severity Levels', fontsize=12, fontweight='bold')
ax2.set_xlim(0, 5)
ax2.grid(axis='x', alpha=0.3)

# 3. Standard application frequency
std_application = {'ISO 26262': 0, 'ISO 21448': 0, 'ISO/SAE 21434': 0}
for _, hazard in aeb_hazards:
    for std in hazard.analyses:
        std_application[std] += 1

ax3.pie(std_application.values(), labels=std_application.keys(), autopct='%1.0f%%',
        colors=['blue', 'green', 'red'], startangle=90, explode=(0.05, 0.05, 0.05))
ax3.set_title('Which Standards Are Used Most in AEB?', fontsize=12, fontweight='bold')

# 4. Requirement counts (simulated based on severity)
requirements = []
for hazard_id, hazard_obj in aeb_hazards:
    severity = severity_values.get(hazard_obj.integrate().get('Integrated Severity', 'LOW'), 1)
    # Higher severity = more requirements (simulated)
    req_count = severity * 15 * len(hazard_obj.analyses)
    requirements.append(req_count)

ax4.bar(hazard_ids, requirements, color='steelblue', alpha=0.7, edgecolor='black')
ax4.set_ylabel('Estimated Safety Requirements', fontsize=11, fontweight='bold')
ax4.set_xlabel('Hazard ID', fontsize=11, fontweight='bold')
ax4.set_title('Safety Requirements per Hazard (Estimated)', fontsize=12, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ AEB comprehensive analysis complete!")
print(f"\nKey Finding: Hazard H4 requires ALL THREE standards - most complex!")

---

## 5. Conflict Resolution Between Standards

Sometimes different standards give **conflicting guidance**. Here's how to resolve conflicts systematically.

### Common Conflicts

| Conflict Type | ISO 26262 Says | ISO 21448 Says | ISO/SAE 21434 Says | Resolution |
|--------------|----------------|----------------|-------------------|------------|
| **Test Coverage** | Code coverage > 95% | Edge case coverage > 95% | Penetration testing required | Do ALL three |
| **Update Frequency** | Avoid updates (stability) | Frequent updates (new scenarios) | Security patches ASAP | Risk-based decision |
| **Fail-Safe vs. Fail-Operational** | Fail-safe (stop immediately) | Fail-operational (continue with degraded mode) | Fail-secure (prevent attack) | Context-dependent |
| **Data Collection** | Minimal (privacy, simplicity) | Maximal (learn edge cases) | Encrypted + anonymized | Balance with strong security |

### Resolution Strategy

```
1. IDENTIFY CONFLICT
 ↓
2. ANALYZE ROOT CAUSE
 - Different risk models?
 - Different assumptions?
 - Different objectives?
 ↓
3. CONSULT STANDARD PRIORITIES
 - Safety > Security > Performance (general rule)
 - Context matters (e.g., parking vs. highway)
 ↓
4. DOCUMENT RATIONALE
 - Why this resolution?
 - What trade-offs?
 - Who approved?
 ↓
5. IMPLEMENT & VERIFY
 - Test that resolution satisfies all standards
 - May need custom solution
```

### Golden Rule: **When in doubt, choose the more conservative approach**

In [ ]:
# Example: Conflict resolution decision matrix
conflicts = [
    {
        "Scenario": "OTA Software Update",
        "ISO 26262 Concern": "Untested code may introduce bugs",
        "ISO 21448 Concern": "Need updates to handle new scenarios",
        "ISO 21434 Concern": "Security patches must be deployed ASAP",
        "Resolution": "Staged rollout with monitoring + emergency rollback",
        "Priority": "Security > New Scenarios > Stability"
    },
    {
        "Scenario": "System Response to Fault",
        "ISO 26262 Concern": "Fail-safe: Stop immediately",
        "ISO 21448 Concern": "Fail-operational: Continue with degraded mode",
        "ISO 21434 Concern": "Fail-secure: Prevent unauthorized access",
        "Resolution": "Context-dependent: Highway=fail-operational, Parking=fail-safe",
        "Priority": "Context determines priority"
    },
    {
        "Scenario": "Data Logging",
        "ISO 26262 Concern": "Minimal logging (reduce complexity)",
        "ISO 21448 Concern": "Extensive logging (learn edge cases)",
        "ISO 21434 Concern": "Encrypted + access control",
        "Resolution": "Log everything, encrypt, anonymize, strict access control",
        "Priority": "Privacy & Security > Learning > Simplicity"
    },
    {
        "Scenario": "Testing Strategy",
        "ISO 26262 Concern": "Code coverage > 95%",
        "ISO 21448 Concern": "Scenario coverage > 95%",
        "ISO 21434 Concern": "Penetration testing",
        "Resolution": "Implement all three test types",
        "Priority": "All are mandatory"
    }
]

df_conflicts = pd.DataFrame(conflicts)

print("=" * 100)
print("CONFLICT RESOLUTION MATRIX")
print("=" * 100)
for idx, row in df_conflicts.iterrows():
    print(f"\n{'='*100}")
    print(f"CONFLICT #{idx+1}: {row['Scenario']}")
    print(f"{'='*100}")
    print(f"ISO 26262:  {row['ISO 26262 Concern']}")
    print(f"ISO 21448:  {row['ISO 21448 Concern']}")
    print(f"ISO 21434:  {row['ISO 21434 Concern']}")
    print(f"\nRESOLUTION: {row['Resolution']}")
    print(f"PRIORITY:   {row['Priority']}")

# Visualize conflict types
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Conflict categories
categories = ['OTA Updates', 'Fault Response', 'Data Logging', 'Testing']
resolution_complexity = [8, 9, 7, 5]  # Subjective complexity score

colors_comp = ['red' if c >= 8 else 'orange' if c >= 6 else 'green' for c in resolution_complexity]

ax1.barh(categories, resolution_complexity, color=colors_comp, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Resolution Complexity (1-10)', fontsize=11, fontweight='bold')
ax1.set_title('Conflict Resolution Complexity', fontsize=12, fontweight='bold')
ax1.set_xlim(0, 10)
ax1.grid(axis='x', alpha=0.3)

# Resolution strategies
strategies = {
    'Do All Three': 2,
    'Context-Dependent': 1,
    'Staged/Balanced': 1
}

ax2.pie(strategies.values(), labels=strategies.keys(), autopct='%1.0f%%',
        colors=['steelblue', 'orange', 'green'], startangle=45,
        explode=(0.05, 0.05, 0.05))
ax2.set_title('Resolution Strategy Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Conflict resolution analysis complete!")
print("\nKey Takeaway: Most conflicts are resolved by combining requirements, not choosing one!")

---

## 6. Documentation and Traceability Requirements

**Integrated safety cases require comprehensive traceability** across all three standards.

### Required Documentation Matrix

| Document Type | ISO 26262 | ISO 21448 | ISO/SAE 21434 | Integration Notes |
|--------------|-----------|-----------|---------------|-------------------|
| **Hazard Analysis** | HARA Table | SOTIF Analysis | TARA Table | Must cross-reference |
| **Safety Requirements** | FSR (Functional Safety Req) | SOTIF Requirements | Security Requirements | Unique IDs, trace to hazards |
| **Architecture** | Safety Architecture | ODD Definition | Security Architecture | Show integration points |
| **V&V Plans** | Safety V&V Plan | SOTIF V&V Plan | Security V&V Plan | Combine test cases |
| **Test Results** | FuSa Test Reports | Edge Case Tests | Penetration Tests | Link to requirements |
| **Safety Case** | Safety Argument | SOTIF Argument | Security Argument | Master safety case |

### Traceability Chain

```
Hazard → Safety Goal → Safety Requirement → Architecture → Implementation → Test → Verification
 ↓ ↓ ↓ ↓ ↓ ↓ ↓
 H-001 SG-001 SR-001 ARCH-001 CODE-001 TC-001 VR-001
 │ │ │ │ │ │ │
 └─────────┴──────────────┴────────────────────┴───────────────┴───────────┴────────┘
 FULL TRACEABILITY MATRIX
```

### Key Traceability Requirements

1. **Bidirectional:** Trace UP (code → hazard) and DOWN (hazard → code)
2. **Cross-Standard:** Link related items across ISO 26262, 21448, and 21434
3. **Version Controlled:** Track changes over time
4. **Tool-Supported:** Manual traceability doesn't scale

In [ ]:
# Example: Traceability matrix for integrated safety
traceability_data = [
    {
        "Hazard ID": "H-001",
        "Hazard": "AEB fails to brake",
        "Standards": "26262, 21448, 21434",
        "Safety Goal": "SG-001",
        "Requirements": "SR-001, SR-002, SR-003",
        "Architecture": "ARCH-001",
        "Implementation": "CODE-001, CODE-002",
        "Test Cases": "TC-001, TC-002, TC-003",
        "Status": "Complete"
    },
    {
        "Hazard ID": "H-002",
        "Hazard": "Cannot detect in fog",
        "Standards": "21448",
        "Safety Goal": "SG-002",
        "Requirements": "SR-004, SR-005",
        "Architecture": "ARCH-002",
        "Implementation": "CODE-003",
        "Test Cases": "TC-004, TC-005",
        "Status": "In Progress"
    },
    {
        "Hazard ID": "H-003",
        "Hazard": "Radar spoofing attack",
        "Standards": "21434",
        "Safety Goal": "SG-003",
        "Requirements": "SR-006",
        "Architecture": "ARCH-003",
        "Implementation": "CODE-004",
        "Test Cases": "TC-006",
        "Status": "Complete"
    },
    {
        "Hazard ID": "H-004",
        "Hazard": "Combined: HW fail + Rain + GPS jam",
        "Standards": "26262, 21448, 21434",
        "Safety Goal": "SG-004",
        "Requirements": "SR-007, SR-008, SR-009, SR-010",
        "Architecture": "ARCH-004, ARCH-005",
        "Implementation": "CODE-005, CODE-006, CODE-007",
        "Test Cases": "TC-007, TC-008, TC-009, TC-010",
        "Status": "Under Review"
    }
]

df_trace = pd.DataFrame(traceability_data)

print("=" * 120)
print("INTEGRATED TRACEABILITY MATRIX")
print("=" * 120)
display(df_trace)

# Visualize traceability coverage
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Status distribution
status_counts = df_trace['Status'].value_counts()
colors_status = {'Complete': 'green', 'In Progress': 'orange', 'Under Review': 'yellow'}
bar_colors = [colors_status.get(s, 'gray') for s in status_counts.index]

ax1.pie(status_counts.values(), labels=status_counts.index, autopct='%1.0f%%',
        colors=bar_colors, startangle=90, explode=[0.05]*len(status_counts))
ax1.set_title('Traceability Status Distribution', fontsize=12, fontweight='bold')

# Requirements per hazard
hazard_ids = df_trace['Hazard ID'].tolist()
req_counts = [len(r.split(', ')) for r in df_trace['Requirements']]

ax2.bar(hazard_ids, req_counts, color='steelblue', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Hazard ID', fontsize=11, fontweight='bold')
ax2.set_ylabel('Number of Requirements', fontsize=11, fontweight='bold')
ax2.set_title('Requirements per Hazard', fontsize=12, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Traceability matrix created!")
print(f"\nCoverage Statistics:")
print(f"  Total Hazards: {len(df_trace)}")
print(f"  Total Requirements: {sum(req_counts)}")
print(f"  Complete: {len(df_trace[df_trace['Status'] == 'Complete'])}")
print(f"  Traceability Coverage: {len(df_trace[df_trace['Status'] == 'Complete']) / len(df_trace) * 100:.1f}%")

---

## 7. Tool Chain for Multi-Standard Compliance

**Integrated compliance requires a coordinated tool chain.** Manual management is impossible at scale.

### Recommended Tool Categories

| Tool Category | Purpose | Examples | Standards Supported |
|--------------|---------|----------|-------------------|
| **Requirements Management** | Track and trace requirements | DOORS, Polarion, Jama | All three |
| **Safety Analysis** | HARA, SOTIF, TARA automation | Medini Analyze, ANSYS medini | ISO 26262, ISO 21448 |
| **Security Analysis** | Threat modeling, vulnerability scanning | Threat Modeler, Veracode | ISO/SAE 21434 |
| **Model-Based Design** | Safety architecture modeling | MATLAB/Simulink, SCADE | ISO 26262, ISO 21448 |
| **Static Analysis** | Code quality and security | Polyspace, Coverity, SonarQube | ISO 26262, ISO 21434 |
| **Dynamic Testing** | Runtime verification | VectorCAST, TPT, CANoe | All three |
| **Scenario Testing** | Edge case and ODD testing | CARLA, LGSVL, IPG CarMaker | ISO 21448 |
| **Penetration Testing** | Security validation | Metasploit, Burp Suite | ISO 21434 |
| **Traceability** | End-to-end linking | Jama, Polarion, CodeBeamer | All three |

### Integrated Tool Chain Architecture

```
┌─────────────────────────────────────────────────────────────┐
│ REQUIREMENTS MANAGEMENT │
│ (Central Hub for All Standards) │
└────────────────────┬────────────────────────────────────────┘
 │
 ┌────────────┼────────────┐
 │ │ │
 ▼ ▼ ▼
┌──────────────┐ ┌──────────────┐ ┌──────────────┐
│ ISO 26262 │ │ ISO 21448 │ │ ISO 21434 │
│ Safety Tools │ │ SOTIF Tools │ │ Security Tools│
└──────┬───────┘ └──────┬───────┘ └──────┬───────┘
 │ │ │
 └────────────────┼────────────────┘
 ▼
 ┌───────────────────────┐
 │ INTEGRATION LAYER │
 │ (Custom Scripts + │
 │ API Connections) │
 └───────────┬───────────┘
 ▼
 ┌───────────────────────┐
 │ VERIFICATION & │
 │ VALIDATION TOOLS │
 └───────────────────────┘
```

### Key Integration Points

1. **Bi-directional traceability:** Requirements tool ↔ Analysis tools
2. **Automated reporting:** Pull data from all tools into unified dashboard
3. **Change management:** Track impacts across all standards
4. **Compliance checking:** Automated verification of completeness

---

## 8. Best Practices for Integrated Safety Case

**Creating a compelling integrated safety case** requires more than just combining documents.

### Best Practice #1: Start with the Big Picture

- **Don't:** Analyze each standard in isolation
- **Do:** Map all hazards across all three standards first
- **Why:** Prevents duplicate work and ensures comprehensive coverage

### Best Practice #2: Use Conservative Integration

- **Don't:** Pick the "easiest" severity level
- **Do:** Use the highest severity from any applicable standard
- **Why:** Ensures safety is never compromised

### Best Practice #3: Document Everything

- **Don't:** Rely on tribal knowledge or informal decisions
- **Do:** Create traceability for every decision and rationale
- **Why:** Auditors and regulators will ask "why did you choose this?"

### Best Practice #4: Automate Where Possible

- **Don't:** Maintain spreadsheets manually
- **Do:** Use integrated tool chains with APIs
- **Why:** Human error in manual processes is inevitable

### Best Practice #5: Test Combined Scenarios

- **Don't:** Test each standard's requirements separately
- **Do:** Create test cases for combined failure modes
- **Why:** Real-world hazards rarely respect standard boundaries

### Best Practice #6: Involve All Stakeholders Early

- **Don't:** Safety team works alone, then hands off to security team
- **Do:** Co-located safety, security, and SOTIF teams
- **Why:** Late integration discoveries are expensive to fix

### Best Practice #7: Plan for Maintenance

- **Don't:** Treat compliance as one-time effort
- **Do:** Build processes for updates, patches, and evolution
- **Why:** Standards require maintenance throughout product lifecycle

### Best Practice #8: Learn from Industry

- **Don't:** Reinvent the wheel
- **Do:** Study how other companies integrate standards
- **Why:** Many lessons have already been learned (expensively!)

---

## 9. Summary and Key Takeaways

### The Three Standards Are Complementary, Not Competing

- **ISO 26262** handles random/systematic failures
- **ISO 21448** handles performance limitations
- **ISO/SAE 21434** handles intentional attacks
- **Together** they form a complete safety framework

### Most Hazards Require Multiple Standards

- In our AEB example, 50% of hazards needed 2+ standards
- Complex scenarios (like H4) require all three
- **Never** assume one standard is sufficient without analysis

### Integration Is About Being Conservative

- Use the **highest** severity level
- Combine **all** requirements
- Test **combined** scenarios
- When in doubt, be **more** conservative

### Key Success Factors

1. **Early integration:** Don't wait until the end to combine standards
2. **Tool support:** Manual integration doesn't scale
3. **Cross-functional teams:** Safety, security, and SOTIF experts must collaborate
4. **Complete traceability:** Every requirement must trace to hazards and tests
5. **Continuous compliance:** Standards compliance is ongoing, not one-time

### Common Pitfalls to Avoid

1. Treating standards as independent
2. Choosing the "easiest" interpretation
3. Insufficient documentation of decisions
4. Late discovery of conflicts
5. Lack of tool integration
6. Testing standards in isolation
7. Treating compliance as a checkbox exercise

### Real-World Impact

**Proper integration:**
- Reduces rework by 40-60%
- Catches 80%+ of integration issues early
- Speeds up audits and certification
- Creates a defensible safety case

**Poor integration:**
- Leads to late-stage redesigns
- Creates gaps in safety coverage
- Increases certification risk
- Results in product delays

### Final Thought

**The goal is not compliance for compliance's sake.** The goal is to build autonomous vehicles that are genuinely safe, secure, and reliable. These three standards, when properly integrated, provide the framework to achieve that goal.

In [ ]:
# Final summary visualization
print("=" * 100)
print("NOTEBOOK 21: STANDARDS INTEGRATION - COMPLETION SUMMARY")
print("=" * 100)

# Summary statistics
summary_stats = {
    'Topics Covered': [
        '1. Three Standards Overview',
        '2. Decision Tree for Standard Selection',
        '3. Combined HARA+SOTIF+TARA Workflow',
        '4. AEB Real-World Example',
        '5. Conflict Resolution Strategies',
        '6. Documentation & Traceability',
        '7. Tool Chain Integration',
        '8. Best Practices',
        '9. Summary & Key Takeaways'
    ],
    'Code Examples': 8,
    'Visualizations': 12,
    'Practical Exercises': 4,
    'Real-World Examples': 1
}

print(f"\nTopics Covered: {len(summary_stats['Topics Covered'])}")
for topic in summary_stats['Topics Covered']:
    print(f"  - {topic}")

print(f"\nInteractive Components:")
print(f"  - Code Examples: {summary_stats['Code Examples']}")
print(f"  - Visualizations: {summary_stats['Visualizations']}")
print(f"  - Practical Exercises: {summary_stats['Practical Exercises']}")
print(f"  - Real-World Case Studies: {summary_stats['Real-World Examples']}")

# Create final summary visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Standards integration pyramid
ax1 = fig.add_subplot(gs[0, :])
standards = ['ISO 26262\n(Functional Safety)', 'ISO 21448\n(SOTIF)', 'ISO/SAE 21434\n(Cybersecurity)']
importance = [100, 100, 100]
colors_std = ['blue', 'green', 'red']
bars = ax1.barh(standards, importance, color=colors_std, alpha=0.6, edgecolor='black', linewidth=2)
ax1.set_xlim(0, 120)
ax1.set_xlabel('Importance Level', fontsize=12, fontweight='bold')
ax1.set_title('All Three Standards Are Equally Critical', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for bar in bars:
    width = bar.get_width()
    ax1.text(width + 2, bar.get_y() + bar.get_height()/2, 
             'REQUIRED', ha='left', va='center', fontweight='bold', fontsize=10)

# 2. Integration maturity levels
ax2 = fig.add_subplot(gs[1, 0])
maturity_levels = ['None', 'Basic', 'Intermediate', 'Advanced', 'Expert']
maturity_scores = [0, 25, 50, 75, 100]
colors_maturity = ['red', 'orange', 'yellow', 'lightgreen', 'darkgreen']
ax2.bar(maturity_levels, maturity_scores, color=colors_maturity, alpha=0.7, edgecolor='black')
ax2.axhline(y=50, color='blue', linestyle='--', linewidth=2, label='Minimum Target')
ax2.set_ylabel('Capability (%)', fontsize=10, fontweight='bold')
ax2.set_title('Integration Maturity Levels', fontsize=11, fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 3. Benefits of proper integration
ax3 = fig.add_subplot(gs[1, 1])
benefits = ['Rework\nReduction', 'Early Issue\nDetection', 'Audit\nSpeed', 'Safety Case\nQuality']
benefit_values = [50, 80, 60, 90]
ax3.bar(benefits, benefit_values, color='steelblue', alpha=0.7, edgecolor='black')
ax3.set_ylabel('Improvement (%)', fontsize=10, fontweight='bold')
ax3.set_title('Benefits of Proper Integration', fontsize=11, fontweight='bold')
ax3.set_ylim(0, 100)
ax3.grid(axis='y', alpha=0.3)
plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 4. Common pitfalls
ax4 = fig.add_subplot(gs[1, 2])
pitfalls = ['Isolation', 'Late\nIntegration', 'Manual\nProcess', 'Poor\nDocs']
pitfall_frequency = [70, 65, 80, 75]
ax4.bar(pitfalls, pitfall_frequency, color='salmon', alpha=0.7, edgecolor='black')
ax4.set_ylabel('Frequency (%)', fontsize=10, fontweight='bold')
ax4.set_title('Common Pitfalls to Avoid', fontsize=11, fontweight='bold')
ax4.set_ylim(0, 100)
ax4.grid(axis='y', alpha=0.3)
plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 5. Key takeaways
ax5 = fig.add_subplot(gs[2, :])
ax5.axis('off')
takeaways_text = """
KEY TAKEAWAYS FROM THIS NOTEBOOK:

1. INTEGRATION IS MANDATORY: ISO 26262, 21448, and 21434 must work together
2. CONSERVATIVE APPROACH: Always use the highest severity level from any standard
3. EARLY IS BETTER: Start integration from day one, not at the end
4. TOOL SUPPORT: Manual integration doesn't scale - invest in tools
5. CROSS-FUNCTIONAL TEAMS: Safety, security, and SOTIF experts must collaborate
6. COMPLETE TRACEABILITY: Every requirement must trace to hazards and tests
7. TEST COMBINED SCENARIOS: Real hazards don't respect standard boundaries
8. CONTINUOUS COMPLIANCE: Standards require ongoing maintenance

REMEMBER: The goal is not compliance for compliance's sake,
          but building autonomous vehicles that are genuinely safe, secure, and reliable.
"""
ax5.text(0.05, 0.95, takeaways_text, transform=ax5.transAxes, fontsize=10,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8, edgecolor='black', linewidth=2))

plt.suptitle('Standards Integration: Complete Framework Summary', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\n" + "=" * 100)
print("CONGRATULATIONS! You have completed Notebook 21: Standards Integration")
print("=" * 100)
print("\nYou now understand how to:")
print("  - Determine which standards apply to any given hazard")
print("  - Perform integrated HARA + SOTIF + TARA analysis")
print("  - Resolve conflicts between standards")
print("  - Create comprehensive traceability matrices")
print("  - Build an integrated safety case")
print("  - Apply best practices for multi-standard compliance")
print("\nNext Steps:")
print("  - Practice applying these techniques to your own AV systems")
print("  - Study real-world safety cases from industry")
print("  - Explore advanced tool chains for automation")
print("  - Continue learning about emerging standards and regulations")
print("\n" + "=" * 100)
print("Thank you for completing this comprehensive workshop on Standards Integration!")
print("=" * 100)